In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
import math
from sklearn.preprocessing import StandardScaler
import random
import sklearn
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [ ]:
# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
maxwell=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\maxwell_dataset.csv",header=None)
columns_maxwell=['Syear','App','Har','Dba','Ifc','Source','Telonuse','Nlan','T01','T02','T03','T04','T05','T06','T07','T08','T09','T10','T11','T12','T13','T14','T15','Duration','Size','Time','Effort']
maxwell.set_axis(columns_maxwell,axis='columns',inplace=True)
maxwell.set_axis(range(1,63),axis=0 ,inplace=True)
maxwell.rename_axis("Features", axis=1,inplace=True)
maxwell.rename_axis("Projects", axis=0,inplace=True)
dataset = maxwell

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.function(reduce_retracing=True)
# Split data into train and test sets

# Create a classifier
estimator = SVR(kernel='linear')

# Create SFS model
sfs = SequentialFeatureSelector(estimator,n_features_to_select='auto', direction='forward', cv=5 , scoring='neg_mean_absolute_error')

# Fit SFS model on training data
sfs.fit(X, y)

# Transform the data to selected features
X_selected = sfs.transform(X)
print(sfs.get_feature_names_out())

#=====================================================================================================================

scaler_X = StandardScaler()
X_selected = scaler_X.fit_transform(X_selected)


# Define the ANN model to be optimized.
def ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=int(neurons_input), activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(int(num_hidden_layers)):
        model.add(tf.keras.layers.Dense(units=int(neurons_hidden), activation='relu'))

    model.add(tf.keras.layers.Dense(units=1))  # Output layer with a single unit for regression.

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

# Define the search space for Bayesian optimization.
pbounds = {
    'neurons_input': (10, 50),
    'neurons_hidden': (10, 50),
    'num_hidden_layers': (1, 5),
    'learning_rate': (1e-5, 1e-2),
    'batch_size': (8, 32),
    'epochs': (5, 20)
}

# Define the function to optimize (minimize RMSE).
def optimize_effort_estimation(neurons_input, neurons_hidden, num_hidden_layers, learning_rate, batch_size, epochs):
    model = ann_model(neurons_input, neurons_hidden, num_hidden_layers, learning_rate)

    model.fit(X_train, y_train, batch_size=int(batch_size), epochs=int(epochs), verbose=0)

    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return -rmse  # Minimize the negative RMSE for Bayesian optimization.

num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_MAE=[]
mean_MMRE=[]
mean_RMSE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Perform Bayesian optimization.
    optimizer = BayesianOptimization(
        f=optimize_effort_estimation,
        pbounds=pbounds,
        random_state=42,
        verbose=2,
    )

    optimizer.maximize(init_points=10, n_iter=30)  # Adjust the number of initial points and iterations.

    # Print the best hyperparameters found.
    best_params = optimizer.max['params']
        
    #build the model
    model = ann_model(best_params['neurons_input'],best_params['neurons_hidden'],best_params['num_hidden_layers'],best_params['learning_rate'])
    # Train the model.
    model.fit(X_train, y_train, epochs=int(best_params['epochs']), batch_size=int(best_params['batch_size']), verbose=0)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)

    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
    
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")   
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")

['x1' 'x2' 'x4' 'x5' 'x6' 'x11' 'x14' 'x16' 'x17' 'x18' 'x20' 'x23' 'x24']
|   iter    |  target   | batch_... |  epochs   | learni... | neuron... | neuron... | num_hi... |
-------------------------------------------------------------------------------------------------
1/1 [==============================] - 0s 104ms/step
| 1         | -1.993e+0 | 16.99     | 19.26     | 0.007323  | 33.95     | 16.24     | 1.624     |
1/1 [==============================] - 0s 61ms/step
| 2         | -9.885e+0 | 9.394     | 17.99     | 0.006015  | 38.32     | 10.82     | 4.88      |
1/1 [==============================] - 0s 66ms/step
| 3         | -2.006e+0 | 27.98     | 8.185     | 0.001826  | 17.34     | 22.17     | 3.099     |
1/1 [==============================] - 0s 61ms/step
| 4         | -2.005e+0 | 18.37     | 9.368     | 0.006122  | 15.58     | 21.69     | 2.465     |
1/1 [==============================] - 0s 54ms/step
| 5         | -2.005e+0 | 18.95     | 16.78     | 0.002005  | 30.57     | 33

1/1 [==============================] - 0s 83ms/step
| 32        | -1.064e+0 | 9.144     | 13.59     | 0.01      | 13.7      | 37.81     | 5.0       |
1/1 [==============================] - 0s 91ms/step
| 33        | -1.057e+0 | 10.85     | 16.32     | 0.01      | 15.06     | 39.87     | 5.0       |
1/1 [==============================] - 0s 97ms/step
| 34        | -1.005e+0 | 8.0       | 20.0      | 0.01      | 12.75     | 42.54     | 5.0       |
1/1 [==============================] - 0s 98ms/step
| 35        | -9.904e+0 | 8.0       | 18.72     | 0.01      | 16.94     | 41.03     | 5.0       |
1/1 [==============================] - 0s 409ms/step
| 36        | -1.089e+0 | 8.802     | 19.71     | 0.007807  | 16.52     | 46.89     | 3.392     |
1/1 [==============================] - 0s 82ms/step
| 37        | -1.034e+0 | 8.0       | 18.51     | 0.01      | 11.43     | 47.24     | 5.0       |
1/1 [==============================] - 0s 75ms/step
| 38        | -1.063e+0 | 10.66     | 14.87    

1/1 [==============================] - 0s 81ms/step
| 29        | -1.261e+0 | 8.34      | 5.343     | 0.01      | 13.8      | 41.93     | 5.0       |
1/1 [==============================] - 0s 74ms/step
| 30        | -1.21e+04 | 8.087     | 19.22     | 0.001273  | 33.85     | 14.28     | 4.481     |
1/1 [==============================] - 0s 74ms/step
| 31        | -1.345e+0 | 10.32     | 6.796     | 0.007025  | 15.71     | 32.39     | 4.289     |
1/1 [==============================] - 0s 81ms/step
| 32        | -4.535e+0 | 10.57     | 12.61     | 0.01      | 14.18     | 36.2      | 5.0       |
1/1 [==============================] - 0s 82ms/step
| 33        | -6.011e+0 | 14.25     | 19.97     | 0.01      | 18.73     | 33.55     | 5.0       |
1/1 [==============================] - 0s 69ms/step
| 34        | -5.421e+0 | 8.092     | 12.95     | 0.005003  | 24.49     | 37.7      | 3.55      |
1/1 [==============================] - 0s 82ms/step
| 35        | -5.151e+0 | 8.0       | 13.09     

1/1 [==============================] - 0s 81ms/step
| 26        | -2.453e+0 | 8.148     | 16.19     | 0.01      | 15.94     | 33.99     | 5.0       |
1/1 [==============================] - 0s 87ms/step
| 27        | -2.754e+0 | 8.0       | 15.81     | 0.01      | 15.57     | 38.62     | 5.0       |
1/1 [==============================] - 0s 81ms/step
| 28        | -3.14e+03 | 8.0       | 18.16     | 0.009296  | 35.11     | 10.0      | 5.0       |
1/1 [==============================] - 0s 80ms/step
| 29        | -2.528e+0 | 8.0       | 11.87     | 0.01      | 14.16     | 38.39     | 5.0       |
1/1 [==============================] - 0s 82ms/step
| 30        | -2.922e+0 | 10.6      | 14.36     | 0.006921  | 37.98     | 10.0      | 5.0       |
1/1 [==============================] - 0s 56ms/step
| 31        | -4.48e+03 | 8.0       | 12.85     | 0.01      | 15.52     | 39.77     | 1.558     |
1/1 [==============================] - 0s 86ms/step
| 32        | -4.757e+0 | 9.216     | 16.28     

1/1 [==============================] - 0s 55ms/step
| 23        | -7.122e+0 | 16.34     | 15.16     | 0.004878  | 13.67     | 10.61     | 1.883     |
1/1 [==============================] - 0s 70ms/step
| 24        | -4.123e+0 | 23.59     | 18.05     | 0.005597  | 23.23     | 48.02     | 3.229     |
1/1 [==============================] - 0s 58ms/step
| 25        | -7.124e+0 | 20.57     | 7.095     | 0.004777  | 31.34     | 14.44     | 2.192     |
1/1 [==============================] - 0s 74ms/step
| 26        | -7.131e+0 | 13.42     | 6.584     | 0.001587  | 31.42     | 41.7      | 4.011     |
1/1 [==============================] - 0s 64ms/step
| 27        | -6.959e+0 | 13.33     | 14.39     | 0.005921  | 10.51     | 43.01     | 2.079     |
1/1 [==============================] - 0s 58ms/step
| 28        | -6.826e+0 | 10.51     | 16.13     | 0.005838  | 46.93     | 17.55     | 1.793     |
1/1 [==============================] - 0s 58ms/step
| 29        | -7.129e+0 | 29.95     | 6.991     

1/1 [==============================] - 0s 82ms/step
| 20        | -3.785e+0 | 8.89      | 14.69     | 0.01      | 18.5      | 39.54     | 5.0       |
1/1 [==============================] - 0s 81ms/step
| 21        | -4.837e+0 | 11.71     | 20.0      | 0.01      | 15.7      | 37.59     | 5.0       |
1/1 [==============================] - 0s 80ms/step
| 22        | -3.028e+0 | 8.488     | 7.451     | 0.00967   | 24.64     | 38.9      | 4.269     |
1/1 [==============================] - 0s 437ms/step
| 23        | -4.095e+0 | 8.0       | 5.476     | 0.01      | 21.36     | 44.02     | 5.0       |
1/1 [==============================] - 0s 97ms/step
| 24        | -6.973e+0 | 8.0       | 20.0      | 0.01      | 45.29     | 10.09     | 5.0       |
1/1 [==============================] - 0s 56ms/step
| 25        | -1.413e+0 | 9.272     | 11.33     | 0.003267  | 41.17     | 13.77     | 1.216     |
1/1 [==============================] - 0s 72ms/step
| 26        | -1.401e+0 | 9.14      | 16.6     